*Đọc kĩ markdown*

In [138]:
import numpy as np
from numpy import array
import pandas as pd
import matplotlib.pyplot as plt
import string
import os
from PIL import Image
import glob
from pickle import dump, load
from time import time


In [ ]:
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import LSTM, Embedding, TimeDistributed, Dense, RepeatVector,\
                         Activation, Flatten, Reshape, concatenate, Dropout, BatchNormalization
from keras.optimizers import Adam, RMSprop
from keras.layers import Bidirectional
from keras.layers import add
from keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.utils import load_img, img_to_array
from keras.models import Model
from keras import Input, layers
from keras import optimizers
from keras.applications.inception_v3 import preprocess_input
from tensorflow.keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical

In [51]:
path_to_desc = "Flickr8k/Flickr8k.token.txt"

In [52]:
def load_doc(filename):
	# open the file as read only
	file = open(filename, 'r')
	# read all text
	text = file.read()
	# close the file
	file.close()
	return text

doc = load_doc(path_to_desc)

In [53]:
# Lưu caption dưới dạng key value: id_image : ['caption 1', 'caption 2', 'caption 3',' caption 4', 'caption 5']
def load_descriptions(doc):
	mapping = dict()
	# process lines
	for line in doc.split('\n'):
		# split line by white space
		tokens = line.split()
		if len(line) < 2:
			continue
		# take the first token as the image id, the rest as the description
		image_id, image_desc = tokens[0], tokens[1:]
		# extract filename from image id
		image_id = image_id.split('.')[0]
		# convert description tokens back to string
		image_desc = ' '.join(image_desc)
		# create the list if needed
		if image_id not in mapping:
			mapping[image_id] = list()
		# store description
		mapping[image_id].append(image_desc)
	return mapping

descriptions = load_descriptions(doc)

In [47]:
# Hàm tiền xử lý các câu mô tả như: đưa về chữ thường, bỏ dấu, bỏ các chữ số....
def clean_descriptions(descriptions):
	# prepare translation table for removing punctuation
	table = str.maketrans('', '', string.punctuation)
	for key, desc_list in descriptions.items():
		for i in range(len(desc_list)):
			desc = desc_list[i]
			# tokenize
			desc = desc.split()
			# convert to lower case
			desc = [word.lower() for word in desc]
			# remove punctuation from each token
			#desc = [w.translate(table) for w in desc]
			# remove hanging 's' and 'a'
			#desc = [word for word in desc if len(word)>1]
			# remove tokens with numbers in them
			desc = [word for word in desc if word.isalpha()]
			# store as string
			desc_list[i] =  ' '.join(desc)

clean_descriptions(descriptions)

In [83]:
max_len = 0
for key, val in descriptions.items(): 
    for string in val:
        max_len = max(max_len, len(string))

print(max_len)

199


In [145]:
def tokenize(descriptions):
    new_descriptions = []
    for key, val in descriptions.items():
        for string in val:
            new_val = tokenize_function(list(string), tokenizer, 200)
            new_descriptions.append((key, new_val["input_ids"]))
    return new_descriptions

new_descriptions = tokenize(descriptions)


In [146]:
file_path = 'descriptions.pkl'

# Lưu dữ liệu vào file pickle
with open(file_path, 'wb') as file:
    dump(new_descriptions, file)

In [57]:
path_to_train_id_file = 'Flickr8k/Flickr_8k.trainImages.txt'
path_to_val_id_file = 'Flickr8k/Flickr_8k.devImages.txt'
path_to_test_id_file ='Flickr8k/Flickr_8k.testImages.txt'
path_to_image_folder = 'Flickr8k/Flicker8k_Dataset/'

In [69]:
from transformers import BertTokenizerFast

tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')

# Định nghĩa max_length
max_len = 256

# Hàm xử lý dữ liệu
def tokenize_function(en_texts, tokenizer, max_len):
    return tokenizer(en_texts, padding="max_length",
                    truncation=True,
                    max_length=max_len,
                    is_split_into_words=True)


In [147]:
def load_set(filename):
	doc = load_doc(filename)
	dataset = list()
	# process line by line
	for line in doc.split('\n'):
		# skip empty lines
		if len(line) < 1:
			continue
		# get the image identifier
		identifier = line.split('.')[0]
		dataset.append(identifier)
	return set(dataset)

train_id = load_set(path_to_train_id_file)
val_id = load_set(path_to_val_id_file)
test_id = load_set(path_to_test_id_file)

In [14]:
len(train_id)

6000

In [148]:
# Lấy lấy các ảnh jpg trong thư mục chứa toàn bộ ảnh Flickr
all_train_images_in_folder = glob.glob(path_to_image_folder + '*.jpg')

In [149]:
train_images = set(open(path_to_train_id_file, 'r').read().strip().split('\n'))
# Danh sách ấy sẽ lưu full path vào biến train_img
train_img = []
for image in all_train_images_in_folder: # Duyệt qua tất cả các file trong folder
    image_name = os.path.basename(image)
    if image_name in train_images: # Nếu tên file của nó thuộc training set
        train_img.append(image) # Thì thêm vào danh sách ảnh sẽ dùng để train

In [150]:
test_images = set(open(path_to_test_id_file, 'r').read().strip().split('\n'))
test_img = []
for image in all_train_images_in_folder: 
    image_name = os.path.basename(image)
    if image_name in test_images:
        test_img.append(image) 

In [151]:
val_images = set(open(path_to_val_id_file, 'r').read().strip().split('\n'))
val_img = []
for image in all_train_images_in_folder:
    image_name = os.path.basename(image)
    if image_name in val_images:
        val_img.append(image) 

In [153]:
# Hàm đọc mô tả từ file  và Thêm 'startseq', 'endseq' cho chuỗi
def load_clean_descriptions(filename, dataset):
	# load document
	doc = load_doc(filename)
	descriptions = dict()
	for line in doc.split('\n'):
		# split line by white space
		tokens = line.split()
		# split id from description
		image_id, image_desc = tokens[0], tokens[1:]
		# skip images not in the set
		if image_id in dataset:
			# create list
			if image_id not in descriptions:
				descriptions[image_id] = list()
			# wrap description in tokens
			# desc = 'startseq ' + ' '.join(image_desc) + ' endseq'
			desc = ' '.join(image_desc)
			# store
			descriptions[image_id].append(desc)
	return descriptions

train_descriptions = load_clean_descriptions('descriptions.txt', train_id)
val_descriptions = load_clean_descriptions('descriptions.txt', val_id)
test_descriptions = load_clean_descriptions('descriptions.txt', test_id)

FileNotFoundError: [Errno 2] No such file or directory: 'descriptions.txt'

In [23]:
train_descriptions

{'1000268201_693b08cb0e': ['child in pink dress is climbing up set of stairs in an entry way',
  'girl going into wooden building',
  'little girl climbing into wooden playhouse',
  'little girl climbing the stairs to her playhouse',
  'little girl in pink dress going into wooden cabin'],
 '1001773457_577c3a7d70': ['black dog and spotted dog are fighting',
  'black dog and tricolored dog playing with each other on the road',
  'black dog and white dog with brown spots are staring at each other in the street',
  'two dogs of different breeds looking at each other on the road',
  'two dogs on pavement moving toward each other'],
 '1002674143_1b742ab4b8': ['little girl covered in paint sits in front of painted rainbow with her hands in bowl',
  'little girl is sitting in front of large painted rainbow',
  'small girl in the grass plays with fingerpaints in front of white canvas with rainbow on it',
  'there is girl with pigtails sitting in front of rainbow painting',
  'young girl with pi

In [24]:
# Tạo list các training caption
all_train_captions = []
for key, val in train_descriptions.items():
    for cap in val:
        all_train_captions.append(cap)
for key, val in val_descriptions.items():
    for cap in val:
        all_train_captions.append(cap)
for key, val in test_descriptions.items():
    for cap in val:
        all_train_captions.append(cap)       
len(all_train_captions)

40000

In [25]:
# Chỉ lấy các từ xuất hiện trên 1 lần
word_count_threshold = 1
word_counts = {}
nsents = 0
for sent in all_train_captions:
    nsents += 1
    for w in sent.split(' '):
        word_counts[w] = word_counts.get(w, 0) + 1

vocab = [w for w in word_counts if word_counts[w] >= word_count_threshold]

In [26]:
# Tạo từ điển map từ Word sang Index và ngược lại
ixtoword = {}
wordtoix = {}

ix = 1
for w in vocab:
    wordtoix[w] = ix
    ixtoword[ix] = w
    ix += 1

# Tính toán bảng từ vựng
vocab_size = len(ixtoword) + 1 # Thêm 1 cho từ dùng để padding

In [27]:
# Thêm token đặc biệt
start_token = '<START>'
end_token = '<END>'
wordtoix[start_token] = len(wordtoix)+1
wordtoix[end_token] = len(wordtoix)+1
ixtoword[len(wordtoix)-1] = start_token
ixtoword[len(wordtoix)] = end_token

In [28]:
# Chuyển all_train_captions sang dạng số
train_sequences = []  # Lưu trữ các chú thích dưới dạng số
for caption in all_train_captions:
    seq = []
    seq.append(wordtoix[start_token])  # Thêm token bắt đầu
    for word in caption.split(' '):  # Duyệt từng từ trong chú thích
        if word in wordtoix:  # Nếu từ có trong từ điển
            seq.append(wordtoix[word])
        else:
            continue  # Bỏ qua từ không có trong từ điển
    seq.append(wordtoix[end_token])  # Thêm token kết thúc
    train_sequences.append(seq)


In [29]:
def captions_to_sequences(descriptions, wordtoix, start_token="<start>", end_token="<end>"):
    sequences = {}
    for image_id, captions in descriptions.items():
        encoded_captions = []
        for caption in captions:
            # Thêm token bắt đầu và kết thúc
            seq = [wordtoix[start_token]] + \
                  [wordtoix[word] for word in caption.split() if word in wordtoix] + \
                  [wordtoix[end_token]]
            encoded_captions.append(seq)
        sequences[image_id] = encoded_captions
    return sequences


In [30]:
def decoded_texts(seq):
    return ' '.join(ixtoword[ix] for ix in seq if ix not in [8659, 8660])

In [31]:
train_descriptions = captions_to_sequences(train_descriptions, wordtoix,start_token, end_token)
val_descriptions = captions_to_sequences(val_descriptions, wordtoix, start_token, end_token)
test_descriptions = captions_to_sequences(test_descriptions, wordtoix, start_token, end_token)


In [35]:
train_descriptions

{'1000268201_693b08cb0e': [[8659,
   1,
   2,
   3,
   4,
   5,
   6,
   7,
   8,
   9,
   10,
   2,
   11,
   12,
   13,
   8660],
  [8659, 14, 15, 16, 17, 18, 8660],
  [8659, 19, 14, 6, 16, 17, 20, 8660],
  [8659, 19, 14, 6, 21, 10, 22, 23, 20, 8660],
  [8659, 19, 14, 2, 3, 4, 15, 16, 17, 24, 8660]],
 '1001773457_577c3a7d70': [[8659, 25, 26, 27, 28, 26, 29, 30, 8660],
  [8659, 25, 26, 27, 31, 26, 32, 33, 34, 35, 36, 21, 37, 8660],
  [8659, 25, 26, 27, 38, 26, 33, 39, 40, 29, 41, 42, 34, 35, 2, 21, 43, 8660],
  [8659, 44, 45, 9, 46, 47, 48, 42, 34, 35, 36, 21, 37, 8660],
  [8659, 44, 45, 36, 49, 50, 51, 34, 35, 8660]],
 '1002674143_1b742ab4b8': [[8659,
   19,
   14,
   52,
   2,
   53,
   54,
   2,
   55,
   9,
   56,
   57,
   33,
   23,
   58,
   2,
   59,
   8660],
  [8659, 19, 14, 5, 60, 2, 55, 9, 61, 56, 57, 8660],
  [8659,
   62,
   14,
   2,
   21,
   63,
   64,
   33,
   65,
   2,
   55,
   9,
   38,
   66,
   33,
   57,
   36,
   67,
   8660],
  [8659, 68, 5, 14, 33, 69, 60, 

In [32]:
# Lưu train_descriptions
with open('train_descriptions.pkl', 'wb') as train_file:
    dump(train_descriptions, train_file)

# Lưu val_descriptions
with open('val_descriptions.pkl', 'wb') as val_file:
    dump(val_descriptions, val_file)

# Lưu test_descriptions
with open('test_descriptions.pkl', 'wb') as test_file:
    dump(test_descriptions, test_file)


*Run đến đây thôi nha*

In [50]:
# Chuyển thành từng dòng
def to_lines(descriptions):
	all_desc = list()
	for key in descriptions.keys():
		[all_desc.append(d) for d in descriptions[key]]
	return all_desc

In [51]:
# Tính toán độ dài nhất của câu mô tả
def max_length(descriptions):
	lines = to_lines(descriptions)
	return max(len(d.split()) for d in lines)

max_length = max_length(train_descriptions)

In [52]:
# Hàm load ảnh, resize về khích thước mà Inception v3 yêu cầu.
def preprocess(image_path):
    # Resize ảnh về 299x299 làm đầu vào model
    img = load_img(image_path, target_size=(299, 299))
    x = img_to_array(img)
    # Thêm một chiều  nữa
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    return x

In [53]:
# Khởi tạo Model INception v3 để tạo ra feauture cho các ảnh của chúng ta
model = InceptionV3(weights='imagenet')
model_new = Model(model.input, model.layers[-2].output)

In [54]:
# Hàm biến ảnh đầu vào thành vector features (2048, )
def encode(image):
    image = preprocess(image) # preprocess the image
    fea_vec = model_new.predict(image) # Get the encoding vector for the image
    fea_vec = np.reshape(fea_vec, fea_vec.shape[1]) # reshape from (1, 2048) to (2048, )
    return fea_vec

In [55]:
# Lặp một vòng qua các ảnh train và biến hết thành vector features
encoding_train = {}
for img in train_img:
    encoding_train[os.path.basename(img)] = encode(img)
# Lưu image embedding train lại
with open("encoded_train_images.pkl", "wb") as encoded_pickle:
    dump(encoding_train, encoded_pickle)


1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 140ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 121ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 124ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 137ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 129ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 127ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 125ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 147ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 136ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 170ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 185ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 153ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 134ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 151ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 137ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 120ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 135ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 147ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 145ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 160ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 169ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 164ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 151ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 159ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 

In [56]:
# Lặp một vòng qua các ảnh test và biến hết thành vector features
encoding_test = {}
for img in test_img:
    encoding_test[os.path.basename(img)] = encode(img)
with open("encoded_test_images.pkl", "wb") as encoded_pickle:
    dump(encoding_test, encoded_pickle)

train_features = load(open("encoded_train_images.pkl", "rb"))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 113ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 133ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 133ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 117ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 146ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 150ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 184ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 135ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 133ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 117ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 117ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 117ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 117ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 117ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 132ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 157ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 125ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 133ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 134ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 117ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 133ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 117ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 133ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 117ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 133ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 

In [57]:
encoding_val = {}
for img in val_img:
    encoding_val[os.path.basename(img)] = encode(img)
with open("encoded_val_images.pkl", "wb") as encoded_pickle:
    dump(encoding_val, encoded_pickle)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 118ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 117ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 117ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 105ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 117ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 117ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 134ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 166ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 118ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 117ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 100ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 117ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 117ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 116ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 117ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 117ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 126ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 117ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 117ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 166ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 133ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 116ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 117ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 117ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 112ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 

In [58]:
# Tải model Glove để embeding word
glove_dir = ''
embeddings_index = {} # empty dictionary
f = open(os.path.join(glove_dir, 'glove.6B.200d.txt'), encoding="utf-8")

for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

embedding_dim = 200

In [59]:
embedding_dim = 200
# Tạo ma trận embeding cho bảng từ vững, mỗi từ embeding bằng 1 vector 200
embedding_matrix = np.zeros((vocab_size, embedding_dim))

# Lặp qua các từ trong danh sách từ
for word, i in wordtoix.items():
    # Lấy embeding của Glove gán vào embeding vector
    embedding_vector = embeddings_index.get(word)
    # Nếu như không None thì gán vào mảng Maxtrix
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector